# NetworkX Tutorial

In this tutorial we will take a look at ways of combining the analysis tools provided by <a href="https://networkx.github.io/documentation/stable/index.html">NetworkX</a> with the visualization capailities of AlgorithmX.

## Simple Graph

Let's start by creating a simple NetworkX graph. We will use 'add_path' to quickly add both nodes and edges.

In [1]:
import networkx as nx

G = nx.Graph()

G.add_path([1, 2, 3])
G.add_path([4, 2, 5])

print('Nodes:', G.nodes)
print('Edges:', G.edges)

Nodes: [1, 2, 3, 4, 5]
Edges: [(1, 2), (2, 3), (2, 4), (2, 5)]


Now that we have all the data we need, we are ready to give it to AlgorithmX. We'll start by creating a `widget`, which will render our graph. Next, we will obtain a `canvas` interface for manipulating the widget's graphics, which we will finally use to add the nodes and edges of our NetworkX graph.

In [2]:
import algorithmx

canvas = algorithmx.jupyter_canvas()

canvas.nodes(G.nodes).add()
canvas.edges(G.edges).add()

canvas

JupyterClient(events=['{"type": "update", "queue": "default", "data": {"attributes": {"nodes": {"1": {"visible…

So we have our simple graph, but we think it could look a little more interesting. Let's define a custom style for our nodes, and also give each one a different color. We can take advantage of the fact that nearly any argument in AlgorithmX can be passed as a lambda function, making our code much more concise.

In [3]:
canvas = algorithmx.jupyter_canvas()

node_style = {
    'shape': 'rect',
    'size': (20, 12)
}
node_colors = {1: 'red', 2: 'green', 3: 'blue', 4: 'orange', 5: 'purple'}

canvas.nodes(G.nodes).add() \
    .set(node_style) \
    .color(lambda n: node_colors[n])

canvas.edges(G.edges).add()

canvas

JupyterClient(events=['{"type": "update", "queue": "default", "data": {"attributes": {"nodes": {"1": {"visible…

## Random Weighted Directed Graph

NetworkX provides a range of functions for generating graphs, which you can read about <a href="https://networkx.github.io/documentation/stable/reference/generators.html">here</a>. For generating a random graph, we will use the basic `gnp_random_graph` function. By providing a seed, we can ensure that we get the same graph every time (otherwise there is no guarantee of it being connected!).

In [4]:
G = nx.gnp_random_graph(10, 0.3, 138)

canvas = algorithmx.jupyter_canvas()
canvas.nodes(G.nodes).add()
canvas.edges(G.edges).add()

canvas

JupyterClient(events=['{"type": "update", "queue": "default", "data": {"attributes": {"nodes": {"0": {"visible…

Making the graph directed is easy - all we have to do is call `G.to_directed()`, and then tell AlgorithmX that the edges should be rendered with an arrow.

In [5]:
G.to_directed()

canvas = algorithmx.jupyter_canvas()
canvas.nodes(G.nodes).add()
canvas.edges(G.edges).add().directed(True)

canvas

JupyterClient(events=['{"type": "update", "queue": "default", "data": {"attributes": {"nodes": {"0": {"visible…

To make the edges weighted, we will first need to configure a `weight` attribute for each edge in our NetworkX graph. Since our graph is random, we'll make our edge weights random as well. For this we will use the `set_edge_attributes` function with a dictionary comprehension.

In [6]:
from random import randint
nx.set_edge_attributes(G, {e: {'weight': randint(1, 10)} for e in G.edges})

Then, we can display the weight by adding a label to each edge. All the data we need is already contained in our NetworkX graph, and we can retreive it with a convenient lambda fuction.

In [7]:
canvas = algorithmx.jupyter_canvas()

canvas.nodes(G.nodes).add()

canvas.edges(G.edges).add() \
    .directed(True) \
    .label().add() \
        .text(lambda e: G.edges[e]['weight'])

canvas

JupyterClient(events=['{"type": "update", "queue": "default", "data": {"attributes": {"nodes": {"0": {"visible…

## Detailed Graph

Now we are going to create a graph that displays a range of interesting properties. Let's begin by generating a random weighted graph, as before.

In [8]:
G = nx.gnp_random_graph(10, 0.3, 201)
nx.set_edge_attributes(G, {e: {'weight': randint(1, 10)} for e in G.edges})

Next, we will use NetworkX to calculate the graph's <a href="https://networkx.github.io/documentation/stable/reference/algorithms/generated/networkx.algorithms.coloring.greedy_color.html#networkx.algorithms.coloring.greedy_color">coloring</a> and <a hre="https://networkx.github.io/documentation/stable/reference/algorithms/generated/networkx.algorithms.centrality.edge_betweenness_centrality.html#networkx.algorithms.centrality.edge_betweenness_centrality">edge centrality</a>.

In [9]:
coloring = nx.greedy_color(G)
centrality = nx.edge_betweenness_centrality(G, weight='weight', normalized=True)

We can now begin displaying the graph. First, we will add the nodes and assign them a color based on their calculated priority. We happen to know that any graph requires at most 4 different colors, and so we prepare these beforehand.

In [10]:
canvas = algorithmx.jupyter_canvas()

color_priority = {0: 'red', 1: 'orange', 2: 'yellow', 3: 'green'}

canvas.nodes(G.nodes).add() \
    .color(lambda n: color_priority[coloring[n]])

print(coloring)

{4: 0, 2: 1, 3: 2, 0: 1, 1: 2, 6: 0, 8: 1, 7: 2, 9: 2, 5: 0}


Afterwards, we will add the edges. Each one will have two labels; one to display it's weight, and another to display it's calculated centrality.

In [11]:
edge_graphics = canvas.edges(G.edges).add()

formatted_centrality = {k: '{0:.2f}'.format(v) for k, v in centrality.items()}

edge_graphics.label().add() \
    .text(lambda e: G.edges[e]['weight']) \

edge_graphics.label('centrality').add() \
    .color('blue') \
    .text(lambda e: formatted_centrality[e])

print(formatted_centrality)

{(0, 1): '0.24', (0, 3): '0.33', (0, 4): '0.07', (1, 4): '0.05', (1, 8): '0.18', (2, 3): '0.16', (2, 4): '0.23', (2, 5): '0.20', (2, 6): '0.02', (2, 7): '0.15', (3, 4): '0.00', (3, 6): '0.19', (4, 8): '0.09', (4, 9): '0.09', (6, 7): '0.05', (8, 9): '0.11'}


Finally, we can see the whole graph.

In [12]:
canvas

JupyterClient(events=['{"type": "update", "queue": "default", "data": {"attributes": {"nodes": {"0": {"visible…